In [26]:

import numpy as np
import pandas as pd

pms = pd.read_csv('player_match_summary_epl_2015-2016.csv')

'''
## EXERCISE 1 ##
Fill in the calculation below for fantasy points from goals.
'''

pms_fantasy = pms.assign(
    pts_minutes=lambda x: np.where(x['minutes'] > 60, 2, 1),
    pts_goals=lambda x: np.select(
        condlist=[
            x['position'].isin(['Goalkeeper', 'Defender']),
            x['position'] == 'Midfielder',
            x['position'] == 'Forward',
        ],
        choicelist=[6 * x['goals'], 5 * x['goals'], 4 * x['goals']],
        default=0
    ),
    pts_assists=lambda x: 3 * x['assists'],
    pts_clean_sheets=lambda x: np.select(
        condlist=[
            x['position'].isin(['Goalkeeper', 'Defender']),
            x['position'] == 'Midfielder',
        ],
        choicelist=[4 * x['clean_sheets'], x['clean_sheets']],
        default=0   # Forward get no points for clean sheets
    ),
    pts_saves=lambda x: np.floor(x['saves'] / 3),
    pts_penalty_saves=lambda x: 5 * x['penalty_saves'],
    pts_penalty_misses=lambda x: -2 * x['penalty_misses'],
    pts_goals_conceded=lambda x: np.where(
        x['position'].isin(['Goalkeeper', 'Defender']),
        -np.floor(x['goals_conceded'] / 2),
        0
    ),
    pts_yellow_cards=lambda x: -x['yellow_cards'],
    pts_red_cards=lambda x: -3 * x['red_cards'],
    pts_own_goals=lambda x: -2 * x['own_goals'],
    pts=lambda x: x['pts_minutes'] + x['pts_goals'] + x['pts_assists'] + x['pts_clean_sheets'] +
        x['pts_saves'] + x['pts_penalty_saves'] + x['pts_penalty_misses'] +
        x['pts_goals_conceded'] + x['pts_yellow_cards'] + x['pts_red_cards'] + x['pts_own_goals']
)


In [ ]:
import matplotlib.pyplot as plt

'''
## EXERCISE 2 ##
Plot first-half fantasy points vs. second-half fantasy points, and
calculate the split-half correlation in fantasy points within each position.
'''

pts_half_1 = (
    pms_fantasy
        .loc[lambda x: x['match_date'] < '2016-01-01']
        .groupby(['player_id', 'player_name', 'position'])
        .sum(numeric_only=True)
)

pts_half_2 = (
    pms_fantasy
        .loc[lambda x: x['match_date'] >= '2016-01-01']
        .groupby(['player_id', 'player_name', 'position'])
        .sum(numeric_only=True)
)

pts = (
    pd.merge(
        pts_half_1,
        pts_half_2,
        on=['player_id', 'player_name', 'position'],
        how='outer',
        suffixes=['_1', '_2']
    )
        .fillna(0)  # any count missing is zero
)

plt.scatter(x=pts['pts_1'], y=pts['pts_2'])

pts.groupby('position').apply(lambda x: x['pts_1'].corr(x['pts_2']))

In [ ]:
'''
## EXERCISE 3 ##
Calculate the split-half correlation by postion for each subcomponent of fantasy points
(e.g. pts_minutes, pts_goals). What are the strongest and weakest correlations?
'''

pts_columns = [
    'pts_minutes', 'pts_goals', 'pts_assists', 'pts_clean_sheets', 'pts_saves', 'pts_penalty_saves',
    'pts_penalty_misses', 'pts_goals_conceded', 'pts_yellow_cards', 'pts_red_cards', 'pts_own_goals'
]

for col in pts_columns:
    print(col)
    print(pts.groupby('position').apply(lambda x: x[f'{col}_1'].corr(x[f'{col}_2'])))


In [ ]:
'''
## EXERCISE 4 ##
What's a better predictor of second-half goals: first-half goals or first-half xG?
(Or is it some combination of the two?)
'''

In [ ]:
'''
## EXERCISE 5 ##
Using everything you've learned from the previous 4 exercises, can you use linear regression
to build a simple projection for second-half fantasy points based on first-half performance?
'''

In [ ]:
'''
## EXERCISE 6 ##
Can you apply your projection model to the 2023-2024 data to date?
'''